In [1]:
library(tidyverse)
library(rvest)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




# Lecture 11: Web scraping

<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Understand how to import data from online sources by scraping web pages.
</div>

These notes correspond to Chapter 26 of your book.


## Ethics of scraping data online
You should carefully read [Section 26.2](https://r4ds.hadley.nz/webscraping.html#scraping-ethics-and-legalities) of the book concerning various ethical and legal issues surrounding scraping web sites for data. In this class we will only look at large, public web sites like Wikipedia and IMDB, where there is no risk of anything bad happening. However, there are other situations where it may be unethical, or even illegal, to harvest data from a website, even if you are technically able. **As data scientists in the real world, it will be up to you to carefully weigh these concerns before using the tools discussed in today's lecture.**

## Reading data from the Internet
These days, it's increasingly common to pull data from online sources. For example, say I wanted to know the population of European countries. This is [easily found](https://en.wikipedia.org/wiki/Demographics_of_Europe#Population_by_country) on Wikipedia. How can I get these data into R and analyze them?

## How do web pages work?

Web pages are written in a special language called HTML (**H**yper**t**ext **M**arkup **L**anguage). Here is a simple example of some HTML:

    <html>
    <head> 
      <title>Page title</title>
    </head>
    <body>
      <h1 id='first'>A heading</h1>
      <p>Some text &amp; <b>some bold text.</b></p>
      <img src='myimg.png' width='100' height='100'>
    </body>

Web scraping is possible because most web pages have a consistent, hierarchical structure. For example, if I asked you how to navigate to the title of the web page shown above, you would follow the "path"

    html > head > title
    
to arrive at "Page title".

## HTML elements

There are a lot of HTML elements that might contain interesting information. Here are a few of the most common:
- Block tags that denote sections of text: `<h1>` (heading), `<p>` (paragraph), `<ul>`/`<ol>` (un)ordered list, etc.
- `<table>` (a table), `<tr>` (a table row), `<td>` (a table cell), etc.
- Each of these elements can contain attributes such as `id=` or `class=`. For example, `<table id="movies">` is probably a table that contains movie information.

The `rvest` package is used to load a web page and extract elements and tables based on their HTML tags. Let's see how it works by scraping the Wikipedia page mentioned earlier:

In [4]:
europop <- read_html("http://en.wikipedia.org/wiki/Demographics_of_Europe#Population_by_country")

In this page there are many tables:

In [18]:
europop %>% html_elements("table.wikitable") %>% html_table
wiki_table((3))%>%select (1:2)%>%slice(-1)
mutate(Year=as.integer(year)) avg_pop=parse_number(Averagepopulation)

ERROR: ignored

How can we find the correct one? One option is to use our browser to find something that uniquely identifies the table that we want. Alternatively, since there are only about 17, we can just at each table until we find the one we want:

In [ ]:
# find the table that contains the population for each country

## 🤔 Quiz

What's the average population density ($\text{persons}/\text{km}^2$) for countries in Europe?

<ol style="list-style-type: upper-alpha;">
    <li>1234.5</li>
    <li>20000.0</li>
    <li>611.8</li>
    <li>6520.5</li>
    <li>101.1</li>
</ol>



In [23]:
# avg pop density
europop %>% html_elements("table.wikitable") %>% html_table
wiki_table((3))%>% select (dens=4)%>%mutate(dens=parse_number(dens))%>%summarise(mean(dens))


[[1]]
# A tibble: 8 × 2
  Year  `Population(% of world total)`
  <chr> <chr>                         
1 CE 1  34 (15%)                      
2 1000  40 (15%)                      
3 1500  78 (18%)                      
4 1600  112 (20%)                     
5 1700  127 (21%)                     
6 1820  224 (21%)                     
7 1913  498 (28%)                     
8 2000  742 (13%)                     

[[2]]
# A tibble: 31 × 13
   Countr…¹   `1` `1000` `1500` `1600` `1700` `1820` `1870` `1913` `1950` `1973`
   <chr>    <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>
 1 Austria    500    700   2000   2500   2500   3369   4520   6767   6935   7586
 2 Belgium    300    400   1400   1600   2000   3424   5096   7666   8640   9738
 3 Denmark    180    360    600    650    700   1155   1888   2983   4269   5022
 4 Finland     20     40    300    400    400   1169   1754   3027   4009   4666
 5 France    5000   6500  15000  18500  21471  31246  38440  41463  41836  52118
 6 Germany   3000   3500  12000  16000  15000  24905  39231  65058  68371  78956
 7 Italy     7000   5000  10500  13100  13300  20176  27888  37248  47105  54751
 8 the Net…   200    300    950   1500   1900   2355   3615   6164  10114  13438
 9 Norway     100    200    300    400    500    970   1735   2447   3265   3961
10 Sweden     200    400    550    760   1260   2585   4164   5621   7015   8137
# … with 21 more rows, 2 more variables: `1998[6]` <int>, `2020` <int>, and
#   abbreviated variable name ¹​`Country/region`

[[3]]
# A tibble: 20 × 13
   Countr…¹   `1` `1000` `1500` `1600` `1700` `1820` `1870` `1913` `1950` `1973`
   <chr>    <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>
 1 Austria    0.2    0.3    0.5    0.4    0.4    0.3    0.4    0.4    0.3    0.2
 2 Belgium    0.1    0.1    0.3    0.3    0.3    0.3    0.4    0.4    0.3    0.2
 3 Denmark    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.2    0.2    0.1
 4 Finland    0      0      0.1    0.1    0.1    0.1    0.1    0.2    0.2    0.1
 5 France     2.2    2.4    3.4    3.3    3.6    3      3      2.3    1.7    1.3
 6 Germany    1.3    1.3    2.7    2.9    2.5    2.4    3.1    3.6    2.7    2  
 7 Italy      3      1.9    2.4    2.4    2.2    1.9    2.2    2.1    1.9    1.4
 8 Netherl…   0.1    0.1    0.2    0.3    0.3    0.2    0.3    0.3    0.4    0.3
 9 Norway     0      0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
10 Sweden     0.1    0.1    0.1    0.1    0.2    0.2    0.3    0.3    0.3    0.2
11 Switzer…   0.1    0.1    0.1    0.2    0.2    0.2    0.2    0.2    0.2    0.2
12 United …   0.3    0.7    0.9    1.1    1.4    2      2.5    2.5    2      1.4
13 Portugal   0.2    0.2    0.2    0.2    0.3    0.3    0.3    0.3    0.3    0.2
14 Spain      1.9    1.5    1.6    1.5    1.5    1.2    1.3    1.1    1.1    0.9
15 Other      0.9    0.4    0.3    0.3    0.3    0.3    0.4    0.4    0.5    0.4
16 Total W…  10.7    9.5   13.1   13.3   13.5   12.8   14.8   14.6   12.1    9.2
17 Eastern…   2.1    2.4    3.1    3      3.1    3.5    4.1    4.4    3.5    2.8
18 Former …   1.7    2.6    3.9    3.7    4.4    5.3    7      8.7    7.1    6.4
19 Sum       14.5   14.5   20.1   20     21     21.6   25.9   27.7   22.7   18.4
20 World    100    100    100    100    100    100    100    100    100    100  
# … with 2 more variables: `1998` <dbl>, `2018` <chr>, and abbreviated variable
#   name ¹​`Country/region`

[[4]]
# A tibble: 73 × 10
   Year  Averag…¹ Live …² Deaths Natur…³ Crude…⁴ Crude…⁵ Crude…⁶ Total…⁷ Life …⁸
   <chr> <chr>    <chr>   <chr>  <chr>   <chr>   <chr>   <chr>   <chr>   <chr>  
 1 Year  Average… Live b… Deaths Natura… Births  Deaths  Natura… Total … Life e…
 2 1950  549,721… 12,202… 6,473… 5,728,… 22.2    11.8    10.4    2.70    62.8   
 3 1951  554,559… 12,112… 6,609… 5,502,… 21.8    11.9    9.9     2.66    62.8   
 4 1952  559,609… 12,142… 6,265… 5,877,… 21.7    11.2    10.5    2.66    64.0   
 5 1953  565,058… 12,120… 6,220…

ERROR: ignored

## 🤔 Quiz

Use the same page Wikipedia page (Demographics of Europe) to answer the following question:

On average, how many people were born *each day* in Europe between 2010 and 2021 (inclusive)?

<ol style="list-style-type: upper-alpha;">
    <li>90210.10</li>
    <li>23043.97</li>
    <li>7710127</li>
    <li>21123.64</li>
    <li>21109.18</li>
</ol>



In [24]:
# average births per day

wiki_table((3))%>% select(c(1:3))%>% slice(-1) %>% mutate_all(parse_number)

[[1]]
# A tibble: 8 × 2
  Year  `Population(% of world total)`
  <chr> <chr>                         
1 CE 1  34 (15%)                      
2 1000  40 (15%)                      
3 1500  78 (18%)                      
4 1600  112 (20%)                     
5 1700  127 (21%)                     
6 1820  224 (21%)                     
7 1913  498 (28%)                     
8 2000  742 (13%)                     

[[2]]
# A tibble: 31 × 13
   Countr…¹   `1` `1000` `1500` `1600` `1700` `1820` `1870` `1913` `1950` `1973`
   <chr>    <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>
 1 Austria    500    700   2000   2500   2500   3369   4520   6767   6935   7586
 2 Belgium    300    400   1400   1600   2000   3424   5096   7666   8640   9738
 3 Denmark    180    360    600    650    700   1155   1888   2983   4269   5022
 4 Finland     20     40    300    400    400   1169   1754   3027   4009   4666
 5 France    5000   6500  15000  18500  21471  31246  38440  41463  41836  52118
 6 Germany   3000   3500  12000  16000  15000  24905  39231  65058  68371  78956
 7 Italy     7000   5000  10500  13100  13300  20176  27888  37248  47105  54751
 8 the Net…   200    300    950   1500   1900   2355   3615   6164  10114  13438
 9 Norway     100    200    300    400    500    970   1735   2447   3265   3961
10 Sweden     200    400    550    760   1260   2585   4164   5621   7015   8137
# … with 21 more rows, 2 more variables: `1998[6]` <int>, `2020` <int>, and
#   abbreviated variable name ¹​`Country/region`

[[3]]
# A tibble: 20 × 13
   Countr…¹   `1` `1000` `1500` `1600` `1700` `1820` `1870` `1913` `1950` `1973`
   <chr>    <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>
 1 Austria    0.2    0.3    0.5    0.4    0.4    0.3    0.4    0.4    0.3    0.2
 2 Belgium    0.1    0.1    0.3    0.3    0.3    0.3    0.4    0.4    0.3    0.2
 3 Denmark    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.2    0.2    0.1
 4 Finland    0      0      0.1    0.1    0.1    0.1    0.1    0.2    0.2    0.1
 5 France     2.2    2.4    3.4    3.3    3.6    3      3      2.3    1.7    1.3
 6 Germany    1.3    1.3    2.7    2.9    2.5    2.4    3.1    3.6    2.7    2  
 7 Italy      3      1.9    2.4    2.4    2.2    1.9    2.2    2.1    1.9    1.4
 8 Netherl…   0.1    0.1    0.2    0.3    0.3    0.2    0.3    0.3    0.4    0.3
 9 Norway     0      0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
10 Sweden     0.1    0.1    0.1    0.1    0.2    0.2    0.3    0.3    0.3    0.2
11 Switzer…   0.1    0.1    0.1    0.2    0.2    0.2    0.2    0.2    0.2    0.2
12 United …   0.3    0.7    0.9    1.1    1.4    2      2.5    2.5    2      1.4
13 Portugal   0.2    0.2    0.2    0.2    0.3    0.3    0.3    0.3    0.3    0.2
14 Spain      1.9    1.5    1.6    1.5    1.5    1.2    1.3    1.1    1.1    0.9
15 Other      0.9    0.4    0.3    0.3    0.3    0.3    0.4    0.4    0.5    0.4
16 Total W…  10.7    9.5   13.1   13.3   13.5   12.8   14.8   14.6   12.1    9.2
17 Eastern…   2.1    2.4    3.1    3      3.1    3.5    4.1    4.4    3.5    2.8
18 Former …   1.7    2.6    3.9    3.7    4.4    5.3    7      8.7    7.1    6.4
19 Sum       14.5   14.5   20.1   20     21     21.6   25.9   27.7   22.7   18.4
20 World    100    100    100    100    100    100    100    100    100    100  
# … with 2 more variables: `1998` <dbl>, `2018` <chr>, and abbreviated variable
#   name ¹​`Country/region`

[[4]]
# A tibble: 73 × 10
   Year  Averag…¹ Live …² Deaths Natur…³ Crude…⁴ Crude…⁵ Crude…⁶ Total…⁷ Life …⁸
   <chr> <chr>    <chr>   <chr>  <chr>   <chr>   <chr>   <chr>   <chr>   <chr>  
 1 Year  Average… Live b… Deaths Natura… Births  Deaths  Natura… Total … Life e…
 2 1950  549,721… 12,202… 6,473… 5,728,… 22.2    11.8    10.4    2.70    62.8   
 3 1951  554,559… 12,112… 6,609… 5,502,… 21.8    11.9    9.9     2.66    62.8   
 4 1952  559,609… 12,142… 6,265… 5,877,… 21.7    11.2    10.5    2.66    64.0   
 5 1953  565,058… 12,120… 6,220…

ERROR: ignored

In [ ]:
# number of days in 2010--2021

## The Simpsons

The Simpsons is a popular and long-running TV show. How many people still watch the Simpsons? What is their most popular episode?

In [25]:
simpsons <- read_html('https://en.wikipedia.org/wiki/List_of_The_Simpsons_episodes_(season_21–present)')

In [28]:
# parse simpsons
simpsons %>% html_elements("table.wikiepisodetable")%>% .((1)) %>% html_table

ERROR: ignored

## 🤔 Quiz

The episode with the largest number of viewers was **Once Upon a Time in Springfield**. Which episode of the Simpsons had the **smallest** number of viewers?


<ol style="list-style-type: upper-alpha;">
    <li>My Octopus and a Teacher</li>
    <li>Treehouse of Horror XXI</li>
    <li>Marge the Meanie</li>
    <li>The D'oh-cial Network</li>
    <li>The Devil Wears Nada</li>
</ol>



In [ ]:
# smallest number of viewers

## IMDB top movies

Let's consider a well-known table: the [top 250 movies on IMDB](https://www.imdb.com/chart/top/).

In [ ]:
imdb.250 <- read_html("https://www.imdb.com/chart/top/")

In [29]:
# parse imdb
imdbf %>% arrange(year)%>%mutate(top_rating), king = rating == top_rating)%>%
  fliter(king)%>%mutate(delta=lead(yaer)-year)


ERROR: ignored

## Exercise

"The Kid" came out in 1921 and has a rating of 8.2. Another movie that was rated at least as high didn't come out until 1927 (Metropolis), so we could say that The Kid reigned as the #1 film for six years. Metropolis reigned for four years until City Lights (rating 8.4) came out.

Which film reigned for the longest amount of time?

In [ ]:
# longest reign

## Super Bowl TV ratings
We just had the Super Bowl. How have the TV ratings for the Super Bowl changed over the years?

In [ ]:
sbtv <- read_html('https://en.wikipedia.org/wiki/Super_Bowl_television_ratings') %>% html_elements('table') %>% .[[1]] %>% html_table

In [ ]:
# viewers over time

How does this compare with other major sports?

- https://en.wikipedia.org/wiki/World_Series_television_ratings
- https://en.wikipedia.org/wiki/NBA_Finals_television_ratings

In [ ]:
# super bowl vs world series

## Scraping other types of web data

Here are some examples of other types of web data we can scrape:

### The UofM Stats department
Let's say I wanted to make a table of all the [undergraduate stats courses](https://lsa.umich.edu/stats/undergraduate-students/statistics-courses.html) offered by the department. 

In [ ]:
stats <- read_html('https://lsa.umich.edu/stats/undergraduate-students/statistics-courses.html')

How should we extract the data from this web page? We notice from inspecting the page that each course title is a `<b>` (bold) element:

In [ ]:
# extract statistics courses

### Reddit
Let's see how to scrape the [UofM Reddit site](https://old.reddit.com/r/uofm):

In [ ]:
top.reddit <- read_html('https://old.reddit.com/r/uofm/top/?sort=top&t=all')

Let's plot the top scoring posts, when they were posted, and how many votes they have received.

In [ ]:
# top posts on r/uofm